In [23]:
from pymongo import MongoClient
from pprint import pprint
import json

client = MongoClient("localhost", 27017)
client.drop_database("userData")

db = client.userData
with open("people.json") as f:
    file_data = json.load(f)
db.users.insert_many(file_data)


In [24]:
for x in db.users.aggregate(
    [
        {
            "$project": {
                "_id": 0,
                "location": {
                    "type": "Point",
                    "coordinates": [
                        {
                            "$convert": {
                                "input": "$location.coordinates.longitude",
                                "to": "double",
                                "onError": 0.0,
                                "onNull": 0.0,
                            }
                        },
                        {
                            "$convert": {
                                "input": "$location.coordinates.latitude",
                                "to": "double",
                                "onError": 0.0,
                                "onNull": 0.0,
                            }
                        },
                    ],
                },
            }
        },
        {"$out": "transformedLocations"},
    ]
):
    pprint(x)


In [25]:
db.transformedLocations.create_index([("location", "2dsphere")])


'location_2dsphere'

In [26]:
for x in db.transformedLocations.aggregate(
    [
        {
            "$geoNear": {
                "near": {"type": "Point", "coordinates": [-18.4, -42.8]},
                "maxDistance": 100000,
                "distanceField": "distance",
            }
        }
    ]
):
    pprint(x)


{'_id': ObjectId('64570d9eb842f828816fc882'),
 'distance': 26473.52536319881,
 'location': {'coordinates': [-18.5996, -42.6128], 'type': 'Point'}}


In [27]:
client.close()
